# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load data from Part I to create dataframe
weather_map = pd.read_csv('output_data/current_weather_data.csv')
weather_map.dropna(inplace = True) 
weather_map.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cairns,-16.92,145.77,75.20,83,99,3.78,AU,1607878535
1,nalut,30.33,10.85,55.67,44,0,11.34,LY,1607878700
2,cidreira,-30.18,-50.21,79.83,64,3,10.25,BR,1607878700
4,puerto ayora,-0.74,-90.35,73.99,90,94,5.99,EC,1607878701
5,nemuro,43.32,145.57,28.04,88,77,10.96,JP,1607878701


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_map[["Lat", "Lng"]]

# Set the weight value
humidity = weather_map["Humidity"]

In [6]:
# Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Create Figure
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the cities to fit weather conditions.
ideal_weather_data = weather_map.loc[(weather_map["Max Temp"] > 70) & (weather_map["Max Temp"] < 80) & (weather_map["Wind Speed"] < 10) & (weather_map["Cloudiness"] == 0)]
ideal_weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,avarua,-21.21,-159.78,73.40,100,0,3.36,CK,1607878701
16,cape town,-33.93,18.42,72.00,46,0,9.17,ZA,1607878487
138,kruisfontein,-34.00,24.73,73.99,77,0,5.01,ZA,1607878719
348,arica,-18.48,-70.30,75.20,53,0,9.17,CL,1607878788
418,bilma,18.69,12.92,78.84,17,0,9.22,NE,1607878797
463,tessalit,20.20,1.01,77.56,19,0,9.71,ML,1607878809
554,puri,19.80,85.85,71.60,88,0,6.26,IN,1607878845


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Store into variable named hotel_df.
hotel_df = ideal_weather_data

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-8-75727173d3fc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,avarua,-21.21,-159.78,73.40,100,0,3.36,CK,1607878701,
16,cape town,-33.93,18.42,72.00,46,0,9.17,ZA,1607878487,
138,kruisfontein,-34.00,24.73,73.99,77,0,5.01,ZA,1607878719,
348,arica,-18.48,-70.30,75.20,53,0,9.17,CL,1607878788,
418,bilma,18.69,12.92,78.84,17,0,9.22,NE,1607878797,
463,tessalit,20.20,1.01,77.56,19,0,9.71,ML,1607878809,
554,puri,19.80,85.85,71.60,88,0,6.26,IN,1607878845,


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to iterate through the weather
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging with keyword hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing information...Next Location")  

D:\Users\lynel\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Missing information...Next Location
Missing information...Next Location
Missing information...Next Location


In [10]:
#View Hotel Dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,avarua,-21.21,-159.78,73.40,100,0,3.36,CK,1607878701,The Rarotongan Beach Resort & Lagoonarium
16,cape town,-33.93,18.42,72.00,46,0,9.17,ZA,1607878487,Gorgeous George Hotel
138,kruisfontein,-34.00,24.73,73.99,77,0,5.01,ZA,1607878719,
348,arica,-18.48,-70.30,75.20,53,0,9.17,CL,1607878788,Hotel Apacheta
418,bilma,18.69,12.92,78.84,17,0,9.22,NE,1607878797,
463,tessalit,20.20,1.01,77.56,19,0,9.71,ML,1607878809,
554,puri,19.80,85.85,71.60,88,0,6.26,IN,1607878845,"MAYFAIR Heritage, Puri"


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer
fig.add_layer(markers)

# Display Map
fig



Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…